In [37]:
import os
import sys
import pandas as pd
import logging
import openrouteservice
import time


logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))
sys.path.append("../scripts")
from distance_proximity import proximity, travel_time

api_key = "5b3ce3597851110001cf624875d479479cd8485d848de2df20e2e0f8"

ITER = 1
# CALL_RANGE = range(0 + 200 * ITER, 200 + 200 * ITER, 1)
CALL_RANGE = range(1005, 1200, 1)


school_df = pd.read_csv("../data/raw/school.csv", encoding="cp1252")


In [17]:
# # Once off to create column for travel time to schoo
# scraped_df = pd.read_json(
#     "../data/raw/scrape_2022-09-02_04-12-25-567025.json", orient="records"
# )
# scraped_df['school_duration'] = 0
# scraped_df['school_distance'] = 0
# scraped_df.to_csv("../data/curated/scraped_with_school_tt.csv")


In [4]:
scraped_school_df = pd.read_csv("../data/curated/scraped_with_school_tt.csv")


In [38]:
from urllib import request


start_time = time.time()
for i in CALL_RANGE:
    while True:
        try:
            candidate_1, candidate_2 = proximity(
                scraped_school_df.iloc[i, -4],
                scraped_school_df.iloc[i, -3],
                school_df.Y.values.tolist(),
                school_df.X.values.tolist(),
            )

            dis_1, dur_1 = travel_time(
                scraped_school_df.iloc[i, -3],
                scraped_school_df.iloc[i, -4],
                school_df.iloc[candidate_1]["X"],
                school_df.iloc[candidate_1]["Y"],
                api_key,
            )
            dis_2, dur_2 = travel_time(
                scraped_school_df.iloc[i, -3],
                scraped_school_df.iloc[i, -4],
                school_df.iloc[candidate_2]["X"],
                school_df.iloc[candidate_2]["Y"],
                api_key,
            )

            if dur_1 > dur_2:
                scraped_school_df.at[i, "school_duration"] = dur_2 / 60
                scraped_school_df.at[i, "school_distance"] = dis_2

            else:
                scraped_school_df.at[i, "school_duration"] = dur_1 / 60
                scraped_school_df.at[i, "school_distance"] = dis_1

            logging.info(f"Finished property index {i}")
        except Exception as e:
            logging.error(e)
            continue
        break

end_time = time.time()
logging.info(f"Time taken, {(end_time - start_time)/60} mins")

scraped_school_df.to_csv("../data/curated/scraped_with_school_tt.csv")


INFO:root:Finished property index 1005
INFO:root:Finished property index 1006
INFO:root:Finished property index 1007
INFO:root:Finished property index 1008
INFO:root:Finished property index 1009
INFO:root:Finished property index 1010
INFO:root:Finished property index 1011
INFO:root:Finished property index 1012
INFO:root:Finished property index 1013
INFO:root:Finished property index 1014
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/matthewrush/virtualenv/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit e